In [2]:
import os
from pathlib import Path
import re
from PIL import Image
import numpy as np
from tqdm import tqdm

In [3]:
TXT_DIR = Path('/home/macosta/ttmp/primus-data/leipzig-filtered/leipzig-txt/')
SAVE_DIR = Path('/home/macosta/ttmp/primus-data/leipzig-filtered/leipzig-delim-7/')
SAVE_DIR.mkdir(exist_ok=True)

In [82]:
STAFFLINE_A = "0000000000000000000000000000000000000000000000000011100000000000000011100000000000000011100000000000000011100000000000000011100000000000000000000000000000000000000000000000000"
STAFFLINE_B = "0000000000000000000000000000000000000000000000000001000000000000000001000000000000000001000000000000000001000000000000000001000000000000000000000000000000000000000000000000000"
STAFFLINE_PATTERN = re.compile(f"(({STAFFLINE_A}|{STAFFLINE_B}) )+")

In [88]:
def save_whitespaced(from_dir, to_dir, debug=False):
    for file in tqdm(os.listdir(from_dir)):
        if file[-4:] != '.txt':
            continue
        with open(TXT_DIR / file, "r") as f:
            text = f.read()
        if text[-1] != ' ':
            text += ' '
        abridged = STAFFLINE_PATTERN.sub("x", text).strip()
        abridged = re.sub(' x', 'x', abridged)
        abridged = re.sub(' +', '_', abridged)
        final = re.sub('x', ' ', abridged)
        final_tokens = final.split()
#         print(final_tokens)
#         raise Exception()
        for token in final_tokens:
            assert token and token[0] != '_' and token[-1] != '_', f"Token: {token}" and token != STAFFLINE_A and token != STAFFLINE_B
        if debug:
            for token in final_tokens:
                cols = token.split('_')
                arr = np.zeros((len(cols), 175), dtype=np.uint8)
                for i, col in enumerate(cols):
                    arr[i] = [(1-int(x)) for x in col]
                Image.fromarray(arr.T * 255).show()
            return
        savename = to_dir / file
        with open(savename, "w") as f:
            f.write(final)

In [90]:
save_whitespaced(TXT_DIR, SAVE_DIR, debug=False)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 29155/29155 [10:52<00:00, 44.68it/s]
